# Топографическая коррекция

В данном документе производится топографическая коррекция сцен Ландсат.

Для того, чтобы произвести топографическую коррекцию нам необходимы данные по высотам рельефа. Следовательно, нам потребуется импортировать высотные данные. В качестве таких данных возьмем SRTM. (Есть ли на Дальний Восток данные получше?). Таким образом схема работы будет такой:

1. Импорт SRTM в набор данных GRASS.
2. Коррекция всех имеющихся сцен Landsat.


## Импорт STRM

Поскольку данные STRM растпространяются в системе координат широта-долгота, то сначала импортируем данные в область, соответствующую системе координат исходных данных, а затем перепроециуем их в нужную нам проекцию.

In [ ]:
import os
import uuid

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

In [ ]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_ll_location_name()
)

grs.grass.run_command('g.mapset', mapset='srtm', flags='c')

Импортируем данные по высотам:

In [ ]:
HOME_DIR = os.getenv("HOME")
ELEV_DIR = os.path.join(HOME_DIR, 'Data', 'SRTM')

print ELEV_DIR

In [ ]:
basenames = ['srtm_63_03', 'srtm_64_03']
for bs in basenames:
    name = os.path.join(ELEV_DIR, bs+'.tif')
    print bs
    grs.grass.run_command('r.in.gdal', input=name, output=bs, overwrite=True, flags='e')
    

Объединим разные куски SRTM в один для удобства работы, а отдельные части удалим:

In [ ]:
grs.grass.run_command(
    'r.patch', input=','.join(basenames), 
    output='srtm',
    overwrite=True
)
for bs in basenames:
    grs.grass.run_command('g.remove', type='rast', name=bs, flags='f')

Перепроецируем, для этого переключимся в область UTM и создадим отдельный набор данных для хранения высот:

In [ ]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='elevation', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

Установим регион, покрывающий все интересующие нас сцены:

In [ ]:
grs.grass.run_command('g.region', region='all_scenes@landsat')

Перепроецируем:

In [ ]:
grs.grass.run_command('r.proj', location=get_ll_location_name(), mapset='srtm', 
                      input='srtm', output='srtm', overwrite=True)

## Топографическая коррекция

Для топографической коррекци нам нужно знать параметры освещенности в момент съемки: азимут на Солнце и зенитный угол. Эти параметры предоставляются в метаданных к снимку и они были сохранены в файл метаданных в GRASS (см. [функцию импорта Landsat в блокноте](000_Initialization.ipynb)). В дальнейшем эти метаданные были скопированы в растры, очищенные от облачности (в блокнотах этот момент не описан, см. соответствующие команды в history.txt, раздел 2.1.).

Названия очищенных от облачности файлов clean.landsatID_BX (например, clean.LC81130272016087LGN00_B7). Получить доступ к метаданным можно через r.support (правда, его придется разбирать на части).

### Анализ метаданных

Переключаемся в LOCATION с данными Landsat:

In [ ]:
grs.grass.run_command('g.mapset', mapset='landsat')
print grs.grass.read_command('g.mapset', flags='p')

Посмотрим на метаданные одного из растров:

In [ ]:
mapname = "clean.LC81130272016087LGN00_B7"
# Временный файл: 
tempfile = uuid.uuid4().hex

grs.grass.run_command('r.support', map=mapname, savehistory=tempfile)
print open(tempfile).read()

os.unlink(tempfile)


Нас интересуют параметры: 
 * sunaz: азимут Солнца
 * sun_elev: угол, дополнительный к зенитному, т.е. зенитный угол = 90 - sun_elev

Напишем функцию, которая принимает на вход название растра и возвращает пару (sunaz, sun_elev). Помимо очевидного входного параметра (имя растра) функции нужно передать объект-обертку вокруг GRASS.

In [ ]:
def get_sun_position(raster_name, grass):
    tempfile = uuid.uuid4().hex
    try:
        grass.run_command('r.support', map=mapname, savehistory=tempfile)
        for line in open(tempfile).readlines():
            search = 'METADATA: sunaz='
            position = line.find(search)
            if position >= 0:
                sunaz = float(line[position + len(search):])
                
            search = 'METADATA: sun_elev='
            position = line.find(search)
            if position >= 0:
                sun_elev = float(line[position + len(search):])
    except:
        raise
    finally:
        os.unlink(tempfile)
        
    return (sunaz, sun_elev)


get_sun_position(mapname, grs.grass)    

### Топографическая коррекция

Создадим функцию для топографической коррекции. Функция будет принимать на входе список растров для коррекции, модель рельефа и префикс названий выходных растров, в которых будет сохранен результат коррекции. Кроме того, функции нужно передать объект-обертку вокруг GRASS.

In [ ]:
def topo_corr(map_list, elevation, result_prefix, grass):
    
    
    sunaz, sun_elev = get_sun_position(map_list[0], grass)
    zenith = 90 - sun_elev
    
    maps = ','.join(map_list)
    
    tempfile = 'tmp' + uuid.uuid4().hex
    try:
        # first pass: create illumination model
        grass.run_command('i.topo.corr', flags='i', 
                          base=elevation, zenith=zenith, azimuth=sunaz, 
                          output=tempfile, overwrite=True)

        # second pass: apply illumination model
        grass.run_command('i.topo.corr', base=tempfile, 
               input=maps, output=result_prefix, zenith=zenith, method='c-factor')
    finally:
        grass.run_command('g.remove', type='rast', name=tempfile, flags='f')

Пример запуска:

In [ ]:
grs.grass.run_command('g.region', rast='toar_LC81130272016087LGN00_B1')
map_list = ['toar_LC81130272016087LGN00_B1', 'toar_LC81130272016087LGN00_B2', 
            'toar_LC81130272016087LGN00_B3', 'toar_LC81130272016087LGN00_B4']
topo_corr(map_list, 'srtm@elevation', 'topo', grs.grass)

Удалим результаты эксперимента:

In [ ]:
for name in map_list:
    grs.grass.run_command('g.remove', type='rast', name='topo_'+name, flags='f')

#### Коррекция всех сцен

Список сцен получим по растрам QA (удобно использовать: один растр на сцену, уникальный суффикс).

In [ ]:
scenes = grs.grass.list_strings('rast', mapset='landsat', pattern='*_BQA')
# s is something like 'LC81130272014049LGN00_BQA@landsat'
scenes = [s[:-12] for s in scenes]
print scenes[:3]

In [ ]:
bands = ['_B1', '_B2', '_B3', '_B4', '_B5', '_B6', '_B7', '_B8', '_B9', '_B10', '_B11']
toar_prefix = 'toar_'
for s in scenes:
    map_list = [toar_prefix + s + b for b in bands]
    grs.grass.run_command('g.region', rast=map_list[0])
    topo_corr(map_list, 'srtm@elevation', 'topo', grs.grass)